In [1]:
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import re

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras.utils import generic_utils
import configparser
import math
from data_generator import data_generators
from models import vgg as nn
from models import losses as model_losses
from models import roi_helpers
import pandas as pd

import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

2022-12-06 10:24:08.983168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 10:24:09.962320: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-06 10:24:09.962357: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-06 10:24:12.016664: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

ModuleNotFoundError: No module named 'keras.engine.topology'

## 1. Load Configuration File

In [2]:
# load configuration file
config = configparser.ConfigParser()
config.read('training_config.conf')
model_params = {}
    
verbose = eval(config['model_params']['verbose'])

# setting for data augmentation
use_horizontal_flips = eval(config['model_params']['use_horizontal_flips'])
use_vertical_flips = eval(config['model_params']['use_vertical_flips'])
rot_90 = eval(config['model_params']['rot_90'])

# anchor box scales
# Note that if im_size is smaller, anchor_box_scales should be scaled
# Original anchor_box_scales in the paper is [128, 256, 512]
model_params['anchor_box_scales'] = eval(config['model_params']['anchor_box_scales'])

# anchor box ratios
model_params['anchor_box_ratios'] = [[1, 1], [1./math.sqrt(2), 2./math.sqrt(2)], [2./math.sqrt(2), 1./math.sqrt(2)]]

# size to resize the smallest side of the image
# Original setting in paper is 600. Set to 416 in here to save training time
model_params['im_size'] = eval(config['model_params']['im_size'])

# image channel-wise mean to subtract
model_params['img_channel_mean'] = eval(config['model_params']['img_channel_mean'])
model_params['img_scaling_factor'] = eval(config['model_params']['img_scaling_factor'])

# number of ROIs to process at once
num_rois = eval(config['model_params']['num_rois'])

# stride at the RPN (this depends on the network configuration)
model_params['rpn_stride'] = eval(config['model_params']['rpn_stride'])

balanced_classes = False

# scaling the stdev
model_params['std_scaling'] = eval(config['model_params']['std_scaling'])
model_params['classifier_regr_std'] = eval(config['model_params']['classifier_regr_std'])

# overlaps for RPN
model_params['rpn_min_overlap'] = eval(config['model_params']['rpn_min_overlap'])
model_params['rpn_max_overlap'] = eval(config['model_params']['rpn_max_overlap'])

# overlaps for classifier ROIs
model_params['classifier_min_overlap'] = eval(config['model_params']['classifier_min_overlap'])
model_params['classifier_max_overlap'] = eval(config['model_params']['classifier_max_overlap'])

# placeholder for the class mapping, automatically generated by the parser
class_mapping = None

model_path = eval(config['outputs']['model_path'])
model_path_regex = re.match("^(.+)(\.hdf5)$", model_path)
if model_path_regex.group(2) != '.hdf5':
	print('Output weights must have .hdf5 filetype')
	exit(1)

# Input path for weights. If not specified, will try to load default weights provided by keras.
input_weight_path = eval(config['inputs']['input_weight_path'])

# Location to store all the metadata related to the training (to be used when testing).
class_mapping_filename = eval(config['outputs']['class_mapping'])

# get data option out of txt or xml
data_options = eval(config['inputs']['data_options'])

num_epochs = eval(config['model_params']['num_epochs'])

record_path = 'record.csv' # Record data (used to save the losses, classification accuracy and mean average precision)


In [3]:
# check if weight path was passed via command line
if input_weight_path:
	base_net_weights = input_weight_path
else:
	# set the path to weights based on backend and model
	base_net_weights = nn.get_weight_path()

In [4]:
if data_options == 'xml':
	# training data path
	data_path = eval(config['inputs']['data_path'])
	from data_generator.pascal_voc_parser import get_data
	all_imgs, classes_count, class_mapping = get_data(data_path)

elif data_options == 'txt':
	# takes around 15-30 minutes to complete the task- depends on size of data
	train_data_path = eval(config['inputs']['train_annotation'])
	val_data_path = eval(config['inputs']['val_annotation'])
	test_data_path = eval(config['inputs']['test_annotation'])
	from data_generator.simple_parser import get_data
	all_imgs1, classes_count, class_mapping = get_data(test_data_path)
	all_imgs2, classes_count, class_mapping = get_data(val_data_path)
	all_imgs, classes_count, class_mapping = get_data(train_data_path)
	all_imgs = all_imgs + all_imgs1 + all_imgs2

else:
	raise ValueError("data_options must be one of 'xml' or 'txt'")

if 'bg' not in classes_count:
	classes_count['bg'] = 0
	class_mapping['bg'] = len(class_mapping)

# inv_map for converting class index back to class
inv_map = {v: k for k, v in class_mapping.items()}

print('Training images per class:')
pprint.pprint(classes_count)
print(f'Num classes (including bg) = {len(classes_count)}')

# save class mapping
with open(class_mapping_filename, 'wb') as f:
	pickle.dump(class_mapping, f)
	print(f'Class mapping has been written to {class_mapping_filename}, and can be loaded when testing to ensure correct results')


Parsing annotation files
Parsing annotation files
Parsing annotation files
Training images per class:
{'Accordion': 192,
 'Adhesive tape': 49,
 'Aircraft': 306,
 'Airplane': 2658,
 'Alarm clock': 32,
 'Alpaca': 111,
 'Ambulance': 99,
 'Animal': 1510,
 'Ant': 135,
 'Antelope': 368,
 'Apple': 771,
 'Armadillo': 6,
 'Artichoke': 46,
 'Asparagus': 66,
 'Auto part': 909,
 'Axe': 27,
 'Backpack': 444,
 'Bagel': 139,
 'Baked goods': 2334,
 'Balance beam': 52,
 'Ball': 859,
 'Balloon': 1788,
 'Banana': 342,
 'Band-aid': 6,
 'Banjo': 35,
 'Barge': 114,
 'Barrel': 405,
 'Baseball bat': 275,
 'Baseball glove': 500,
 'Bat': 140,
 'Bathroom accessory': 206,
 'Bathroom cabinet': 100,
 'Bathtub': 134,
 'Beaker': 31,
 'Bear': 94,
 'Bed': 609,
 'Bee': 1689,
 'Beehive': 103,
 'Beer': 1177,
 'Beetle': 411,
 'Bell pepper': 192,
 'Belt': 57,
 'Bench': 1051,
 'Bicycle': 4271,
 'Bicycle helmet': 2607,
 'Bicycle wheel': 6624,
 'Bidet': 130,
 'Billboard': 1253,
 'Billiard table': 169,
 'Binoculars': 26,
 'Bird

## Setup Model

In [5]:
input_shape_img = (None, None, 3)

img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(None, 4))

# define the base network
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(model_params['anchor_box_scales']) * len(model_params['anchor_box_ratios'])
rpn = nn.rpn(shared_layers, num_anchors)
# create model for RPN
model_rpn = Model(img_input, rpn[:2])

# define the classifier, built on the base layer and rpn
classifier = nn.classifier(shared_layers, roi_input, num_rois, nb_classes=len(classes_count))
# create model for object classification
model_classifier = Model([img_input, roi_input], classifier)

# this is a model that holds both the RPN and the classifier, used to load/save weights for the models
model_all = Model([img_input, roi_input], rpn[:2] + classifier)

2022-12-02 04:52:26.510483: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-02 04:52:27.157590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-02 04:52:27.157812: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-12-02 04:52:27.157842: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-12-02 04:52:27.159994: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-12-02 04:52:27.160083: I tensorflow/stream_executor/platform/default/d

## Setup Data Generator

In [6]:
train_imgs = [s for s in all_imgs if s['imageset'] == 'train']
val_imgs = [s for s in all_imgs if s['imageset'] == 'val']

random.shuffle(train_imgs)
num_imgs = len(train_imgs)

print(f'Num train samples {len(train_imgs)}')
print(f'Num val samples {len(val_imgs)}')

# define generators for ROI
data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, config, model_params, nn.get_img_output_length, mode='train')
data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, config, model_params, nn.get_img_output_length, mode='val')

Num train samples 200002
Num val samples 35925


## Define Model Parameters

In [7]:
# Because I can only run the session several hours one time (then I need to connect again), 
# I need to save the model and load the model to continue training
if not os.path.isfile(model_path):
    # If this is the begin of the training, load the pre-traind base network such as vgg-16
    try:
        print('This is the first time of your training')
        print('loading weights from {}'.format(base_net_weights))
        model_rpn.load_weights(base_net_weights, by_name=True)
        model_classifier.load_weights(base_net_weights, by_name=True)
    except:
        print('Could not load pretrained model weights. Weights can be found in the keras application folder \
            https://github.com/fchollet/keras/tree/master/keras/applications')
    
    # Create the record.csv file to record losses, acc and mAP
    record_df = pd.DataFrame(columns=['mean_overlapping_bboxes', 'class_acc', 'loss_rpn_cls', 'loss_rpn_regr', 'loss_class_cls', 'loss_class_regr', 'curr_loss', 'elapsed_time', 'mAP'])
else:
    # If this is a continued training, load the trained model from before
    print('Continue training based on previous trained model')
    print('Loading weights from {}'.format(model_path))
    model_rpn.load_weights(model_path, by_name=True)
    model_classifier.load_weights(model_path, by_name=True)
    
    # Load the records
    record_df = pd.read_csv(record_path)

    r_mean_overlapping_bboxes = record_df['mean_overlapping_bboxes']
    r_class_acc = record_df['class_acc']
    r_loss_rpn_cls = record_df['loss_rpn_cls']
    r_loss_rpn_regr = record_df['loss_rpn_regr']
    r_loss_class_cls = record_df['loss_class_cls']
    r_loss_class_regr = record_df['loss_class_regr']
    r_curr_loss = record_df['curr_loss']
    r_elapsed_time = record_df['elapsed_time']
    r_mAP = record_df['mAP']

    print('Already train %dK batches'% (len(record_df)))

optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_rpn.compile(optimizer=optimizer, loss=[model_losses.rpn_loss_cls(num_anchors), model_losses.rpn_loss_regr(num_anchors)])
model_classifier.compile(optimizer=optimizer_classifier, loss=[model_losses.class_loss_cls, model_losses.class_loss_regr(len(classes_count)-1)], metrics={f'dense_class_{len(classes_count)}': 'accuracy'})
model_all.compile(optimizer='sgd', loss='mae')

total_epochs = len(record_df)
r_epochs = len(record_df)
total_epochs += num_epochs

epoch_length = 1000
iter_num = 0

losses = np.zeros((epoch_length, 5))
rpn_accuracy_rpn_monitor = []
rpn_accuracy_for_epoch = []

if len(record_df)==0:
    best_loss = np.Inf
else:
    best_loss = np.min(r_curr_loss.iloc[-10:])

class_mapping_inv = {v: k for k, v in class_mapping.items()}

Continue training based on previous trained model
Loading weights from models/model_vgg16_open_images.hdf5


2022-12-02 04:52:30.287104: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 411041792 exceeds 10% of free system memory.


Already train 274K batches


In [8]:
print('Starting training')
start_time = time.time()
vis = True
for epoch_num in range(num_epochs):

	progbar = generic_utils.Progbar(epoch_length)
	print('Epoch {}/{}'.format(r_epochs + 1, total_epochs))
    
	r_epochs += 1

	while True:
		try:
			if len(rpn_accuracy_rpn_monitor) == epoch_length and verbose:
				mean_overlapping_bboxes = float(sum(rpn_accuracy_rpn_monitor))/len(rpn_accuracy_rpn_monitor)
				rpn_accuracy_rpn_monitor = []
				print(f'\nAverage number of overlapping bounding boxes from RPN = {mean_overlapping_bboxes} for {epoch_length} previous iterations')
				if mean_overlapping_bboxes == 0:
					print('\nRPN is not producing bounding boxes that overlap the ground truth boxes. Check RPN settings or keep training.')

			X, Y, img_data = next(data_gen_train)

			loss_rpn = model_rpn.train_on_batch(X, Y)
			P_rpn = model_rpn.predict_on_batch(X)

			R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], model_params, use_regr=True, max_boxes=300, overlap_thresh=0.7)
			# note: calc_iou converts from (x1,y1,x2,y2) to (x,y,w,h) format
            # X2: bboxes that iou > C.classifier_min_overlap for all gt bboxes in 300 non_max_suppression bboxes
            # Y1: one hot code for bboxes from above => x_roi (X)
            # Y2: corresponding labels and corresponding gt bboxes
			X2, Y1, Y2, IouS = roi_helpers.calc_iou(R, img_data, model_params, class_mapping)

			if X2 is None:
				rpn_accuracy_rpn_monitor.append(0)
				rpn_accuracy_for_epoch.append(0)
				continue

			neg_samples = np.where(Y1[0, :, -1] == 1)
			pos_samples = np.where(Y1[0, :, -1] == 0)

			if len(neg_samples) > 0:
				neg_samples = neg_samples[0]
			else:
				neg_samples = []

			if len(pos_samples) > 0:
				pos_samples = pos_samples[0]
			else:
				pos_samples = []
			
			rpn_accuracy_rpn_monitor.append(len(pos_samples))
			rpn_accuracy_for_epoch.append((len(pos_samples)))

			if num_rois > 1:
				if len(pos_samples) < num_rois//2:
					selected_pos_samples = pos_samples.tolist()
				else:
					selected_pos_samples = np.random.choice(pos_samples, num_rois//2, replace=False).tolist()
				try:
					selected_neg_samples = np.random.choice(neg_samples, num_rois - len(selected_pos_samples), replace=False).tolist()
				except:
					selected_neg_samples = np.random.choice(neg_samples, num_rois - len(selected_pos_samples), replace=True).tolist()

				sel_samples = selected_pos_samples + selected_neg_samples
			else:
				# in the extreme case where num_rois = 1, we pick a random pos or neg sample
				selected_pos_samples = pos_samples.tolist()
				selected_neg_samples = neg_samples.tolist()
				if np.random.randint(0, 2):
					sel_samples = random.choice(neg_samples)
				else:
					sel_samples = random.choice(pos_samples)

			# training_data: [X, X2[:, sel_samples, :]]
            # labels: [Y1[:, sel_samples, :], Y2[:, sel_samples, :]]
            #  X                     => img_data resized image
            #  X2[:, sel_samples, :] => num_rois (4 in here) bboxes which contains selected neg and pos
            #  Y1[:, sel_samples, :] => one hot encode for num_rois bboxes which contains selected neg and pos
            #  Y2[:, sel_samples, :] => labels and gt bboxes for num_rois bboxes which contains selected neg and pos
			loss_class = model_classifier.train_on_batch([X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])

			losses[iter_num, 0] = loss_rpn[1]
			losses[iter_num, 1] = loss_rpn[2]

			losses[iter_num, 2] = loss_class[1]
			losses[iter_num, 3] = loss_class[2]
			losses[iter_num, 4] = loss_class[3]

			progbar.update(iter_num+1, [('rpn_cls', losses[iter_num, 0]), ('rpn_regr', losses[iter_num, 1]),
									  ('detector_cls', losses[iter_num, 2]), ('detector_regr', losses[iter_num, 3])])

			iter_num += 1
			
			if iter_num == epoch_length:
				loss_rpn_cls = np.mean(losses[:, 0])
				loss_rpn_regr = np.mean(losses[:, 1])
				loss_class_cls = np.mean(losses[:, 2])
				loss_class_regr = np.mean(losses[:, 3])
				class_acc = np.mean(losses[:, 4])

				mean_overlapping_bboxes = float(sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)
				rpn_accuracy_for_epoch = []

				if verbose:
					print(f'Mean number of bounding boxes from RPN overlapping ground truth boxes: {mean_overlapping_bboxes}')
					print(f'Classifier accuracy for bounding boxes from RPN: {class_acc}')
					print(f'Loss RPN classifier: {loss_rpn_cls}')
					print(f'Loss RPN regression: {loss_rpn_regr}')
					print(f'Loss Detector classifier: {loss_class_cls}')
					print(f'Loss Detector regression: {loss_class_regr}')
					print(f'Elapsed time: {time.time() - start_time}')
					elapsed_time = (time.time()-start_time)/60

				curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
				iter_num = 0
				start_time = time.time()

				if curr_loss < best_loss:
					if verbose:
						print(f'Total loss decreased from {best_loss} to {curr_loss}, saving weights')
					best_loss = curr_loss
					model_all.save_weights(model_path_regex.group(1) + model_path_regex.group(2))

				new_row = {'mean_overlapping_bboxes':round(mean_overlapping_bboxes, 3), 
                           'class_acc':round(class_acc, 3), 
                           'loss_rpn_cls':round(loss_rpn_cls, 3), 
                           'loss_rpn_regr':round(loss_rpn_regr, 3), 
                           'loss_class_cls':round(loss_class_cls, 3), 
                           'loss_class_regr':round(loss_class_regr, 3), 
                           'curr_loss':round(curr_loss, 3), 
                           'elapsed_time':round(elapsed_time, 3), 
                           'mAP': 0}

				record_df = record_df.append(new_row, ignore_index=True)
				record_df.to_csv(record_path, index=0)
				break

		except Exception as e:
			print(f'Exception: {e}')
			continue

print('Training complete, exiting.')

Starting training
Epoch 275/294


2022-12-02 04:52:32.172855: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-02 04:52:32.344453: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-12-02 04:52:33.512290: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-12-02 04:52:33.588180: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-12-02 04:52:34.376704: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


2994/3000 [============================>.] - ETA: 12s - rpn_cls: 1.9973 - rpn_regr: 0.3534 - detector_cls: 2.3488 - detector_regr: 0.3683
Average number of overlapping bounding boxes from RPN = 16.133333333333333 for 3000 previous iterations
3000/3000 [==============================] - 6464s 2s/step - rpn_cls: 1.9955 - rpn_regr: 0.3534 - detector_cls: 2.3474 - detector_regr: 0.3683
Mean number of bounding boxes from RPN overlapping ground truth boxes: 16.12807717897538
Classifier accuracy for bounding boxes from RPN: 0.5399166666666667
Loss RPN classifier: 1.9954754365913037
Loss RPN regression: 0.3534106462836886
Loss Detector classifier: 2.347391406693806
Loss Detector regression: 0.36829062302814175
Elapsed time: 6463.8295176029205
Epoch 276/294
2983/3000 [============================>.] - ETA: 17s - rpn_cls: 2.0511 - rpn_regr: 0.3505 - detector_cls: 2.3581 - detector_regr: 0.3729
Average number of overlapping bounding boxes from RPN = 16.441333333333333 for 3000 previous iterations

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.arange(0, r_epochs), record_df['mean_overlapping_bboxes'], 'r')
plt.title('mean_overlapping_bboxes')
plt.subplot(1,2,2)
plt.plot(np.arange(0, r_epochs), record_df['class_acc'], 'r')
plt.title('class_acc')

plt.show()

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.arange(0, r_epochs), record_df['loss_rpn_cls'], 'r')
plt.title('loss_rpn_cls')
plt.subplot(1,2,2)
plt.plot(np.arange(0, r_epochs), record_df['loss_rpn_regr'], 'r')
plt.title('loss_rpn_regr')
plt.show()


plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.arange(0, r_epochs), record_df['loss_class_cls'], 'r')
plt.title('loss_class_cls')
plt.subplot(1,2,2)
plt.plot(np.arange(0, r_epochs), record_df['loss_class_regr'], 'r')
plt.title('loss_class_regr')
plt.show()

plt.plot(np.arange(0, r_epochs), record_df['curr_loss'], 'r')
plt.title('total_loss')
plt.show()

# plt.figure(figsize=(15,5))
# plt.subplot(1,2,1)
# plt.plot(np.arange(0, r_epochs), record_df['curr_loss'], 'r')
# plt.title('total_loss')
# plt.subplot(1,2,2)
# plt.plot(np.arange(0, r_epochs), record_df['elapsed_time'], 'r')
# plt.title('elapsed_time')
# plt.show()

# plt.title('loss')
# plt.plot(np.arange(0, r_epochs), record_df['loss_rpn_cls'], 'b')
# plt.plot(np.arange(0, r_epochs), record_df['loss_rpn_regr'], 'g')
# plt.plot(np.arange(0, r_epochs), record_df['loss_class_cls'], 'r')
# plt.plot(np.arange(0, r_epochs), record_df['loss_class_regr'], 'c')
# # plt.plot(np.arange(0, r_epochs), record_df['curr_loss'], 'm')
# plt.show()